In [ ]:
# %pip install pytz
# %pip install geopy
# %pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [7]:

import pandas as pd
import numpy as np
import pandas_gbq as gbq
import requests

from datetime import datetime, timezone
from pytz import timezone as tz

from geopy.geocoders import Nominatim

from google.oauth2 import service_account

import logging
logging.basicConfig(level=logging.INFO)


Table Content
1. extract
2. Transform
3. Load

## 1. Data Extraction

In [8]:
# define base_url
# base url for real time (last 24 hours)
BASE_URL = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson'
#base url for historical
BASE_URL_HISTORICAL = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

In [ ]:
#get data
response = requests.get(BASE_URL)
data = response.json()
data

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [15]:
# convert json to dataframe
records = []

for feature in data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']  # [lon, lat, depth]

    record = {
        "id": feature.get("id"),
        "place": props.get("place"),
        "mag": props.get("mag"),
        "time": pd.to_datetime(props.get("time"), unit='ms'),
        "updated": pd.to_datetime(props.get("updated"), unit='ms'),
        "tz": props.get("tz"),
        "felt": props.get("felt"),
        "cdi": props.get("cdi"),
        "mmi": props.get("mmi"),
        "alert": props.get("alert"),
        "status": props.get("status"),
        "tsunami": props.get("tsunami"),
        "sig": props.get("sig"),
        "net": props.get("net"),
        "code": props.get("code"),
        "ids": props.get("ids"),
        "sources": props.get("sources"),
        "types": props.get("types"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth": coords[2],
        "fetched_at": datetime.now(timezone.utc)
    }
    records.append(record)

records_df = pd.DataFrame(records)
records_df

,id,place,mag,time,updated,tz,felt,cdi,mmi,alert,...,sig,net,code,ids,sources,types,longitude,latitude,depth,fetched_at
0,nc75207327,"2 km E of Pinnacles, CA",1.87,2025-07-09 03:27:19.310,2025-07-09 03:28:54.752,None,NaN,NaN,NaN,None,...,54,nc,75207327,",nc75207327,",",nc,",",nearby-cities,origin,phase-data,",-121.122833,36.535999,9.200000,2025-07-09 03:42:17.383198+00:00
1,nc75207322,"8 km NNW of The Geysers, CA",0.68,2025-07-09 03:22:44.550,2025-07-09 03:24:19.610,None,NaN,NaN,NaN,None,...,7,nc,75207322,",nc75207322,",",nc,",",nearby-cities,origin,phase-data,",-122.815002,38.828499,1.410000,2025-07-09 03:42:17.383741+00:00
2,uw62136042,"22 km ENE of Ashford, Washington",0.36,2025-07-09 03:17:23.080,2025-07-09 03:18:56.910,None,NaN,NaN,NaN,None,...,2,uw,62136042,",uw62136042,",",uw,",",origin,phase-data,",-121.768166,46.846832,-1.720000,2025-07-09 03:42:17.384381+00:00
3,nc75207317,"4 km NNW of The Geysers, CA",0.75,2025-07-09 03:17:09.340,2025-07-09 03:18:44.596,None,NaN,NaN,NaN,None,...,9,nc,75207317,",nc75207317,",",nc,",",nearby-cities,origin,phase-data,",-122.778999,38.805168,2.370000,2025-07-09 03:42:17.384900+00:00
4,ok2025nhoo,"5 km E of Meridian, Oklahoma",1.62,2025-07-09 03:16:14.009,2025-07-09 03:20:12.679,None,NaN,NaN,NaN,None,...,40,ok,2025nhoo,",ok2025nhoo,",",ok,",",origin,phase-data,",-97.184959,35.847298,7.242111,2025-07-09 03:42:17.385164+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,av93661711,"7 km W of Akutan, Alaska",0.64,2025-07-08 04:10:43.050,2025-07-08 21:22:40.670,None,NaN,NaN,NaN,None,...,6,av,93661711,",av93661711,",",av,",",origin,phase-data,",-165.888333,54.143000,4.810000,2025-07-09 03:42:17.465689+00:00
450,us7000qba0,"29 km NNW of Diphu, India",4.20,2025-07-08 03:52:10.603,2025-07-08 04:08:33.040,None,NaN,NaN,NaN,None,...,271,us,7000qba0,",us7000qba0,",",us,",",origin,phase-data,",93.361700,26.102100,10.000000,2025-07-09 03:42:17.465828+00:00
451,nc75206892,"6 km WSW of Gilroy, CA",1.47,2025-07-08 03:52:05.750,2025-07-08 04:12:20.273,None,NaN,NaN,NaN,None,...,33,nc,75206892,",nc75206892,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",-121.622833,36.972668,3.700000,2025-07-09 03:42:17.465971+00:00
452,tx2025nhkbrf,"37 km ESE of Malaga, New Mexico",1.00,2025-07-08 03:47:39.121,2025-07-08 23:58:38.906,None,NaN,NaN,NaN,None,...,15,tx,2025nhkbrf,",tx2025nhkbrf,",",tx,",",origin,phase-data,",-103.725000,32.066000,7.984000,2025-07-09 03:42:17.466112+00:00


## 2. Transformation

- basic cleaning
- data enrichment

In [16]:
df =  records_df.copy()

In [17]:
df = df.drop_duplicates(subset=['id'])

df = df.dropna(subset=["mag", "place", "latitude", "longitude", "time"])

In [18]:
# Convert time columns to datetime if not already
# Convert UTC time columns to datetime with Asia/Jakarta timezone
WIB = tz('Asia/Jakarta')
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['updated'] = pd.to_datetime(df['updated'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['fetched_at'] = pd.to_datetime(df['fetched_at'], errors='coerce').dt.tz_convert('UTC').dt.tz_convert(WIB)

In [19]:
#  Filter only valid magnitudes
df = df[df['mag'] >= 0]

# Select relevant columns
selected_columns = [
    "id", "place", "mag", "time", "updated",
    "latitude", "longitude", "depth",
    "tsunami", "sig", "status", "alert", "types",
    "felt", "cdi", "mmi", "fetched_at"
]
df = df[selected_columns]

In [20]:
# 2b. Data Enrichment

# Add Magnitude Category
def classify_magnitude(mag):
        if mag < 2.0:
            return "Micro"
        elif mag < 4.0:
            return "Minor"
        elif mag < 5.5:
            return "Light"
        elif mag < 7.0:
            return "Moderate"
        elif mag < 8.0:
            return "Strong"
        else:
            return "Major"

df['mag_category'] = df['mag'].apply(classify_magnitude)

In [23]:
# Add address details
def get_address_detail(latitude, longitude):
    result = {}
    coder = Nominatim(user_agent="myGeocoder", timeout=3)
    # print(coder.reverse(f"{latitude}, {longitude}"))
    address = coder.reverse(f"{latitude}, {longitude}")

    result["city"] = address.raw.get("address").get("city") if address else np.nan
    result["state"] = address.raw.get("address").get("state") if address else np.nan
    result["country"] = address.raw.get("address").get("country") if address else np.nan
    return result

address_detail = df.apply(lambda row: get_address_detail(row['latitude'], row['longitude']), axis=1, result_type='expand')
df = pd.concat([df, address_detail], axis=1)

In [24]:
# Add time-based columns
df['day_of_week'] = df['time'].dt.day_name()
df['hour_of_day'] = df['time'].dt.hour

In [26]:
df.to_csv(f"../data/earthquake-{datetime.now().strftime('%Y%m%d')}.csv", index=False)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424 entries, 0 to 453
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype                       
---  ------        --------------  -----                       
 0   id            424 non-null    object                      
 1   place         424 non-null    object                      
 2   mag           424 non-null    float64                     
 3   time          424 non-null    datetime64[ns, Asia/Jakarta]
 4   updated       424 non-null    datetime64[ns, Asia/Jakarta]
 5   latitude      424 non-null    float64                     
 6   longitude     424 non-null    float64                     
 7   depth         424 non-null    float64                     
 8   tsunami       424 non-null    int64                       
 9   sig           424 non-null    int64                       
 10  status        424 non-null    object                      
 11  alert         2 non-null      object                      
 12 

In [28]:
PROJECT_ID = "earthquake-jcds"
TABLE_ID = "earthquake.events"
CREDENTIAL_PATH = "../config/earthquake-jcds-093b281896d5.json"

In [30]:
credentials =  service_account.Credentials.from_service_account_file(CREDENTIAL_PATH)

In [31]:
gbq.to_gbq(
    dataframe=df,
    destination_table=TABLE_ID,
    project_id=PROJECT_ID,
    credentials=credentials,
    if_exists="replace"
)

424 out of 424 rows loaded.<?, ?it/s]INFO:pandas_gbq.gbq_connector:
100%|██████████| 1/1 [00:00<00:00, 1135.44it/s]
